## Import

In [1]:
from ALLCools.clustering import *
from ALLCools.mcds import MCDS
from wmb import brain, aibs, broad, mm10

import pandas as pd
import anndata

import matplotlib.pyplot as plt
from ALLCools.plot import *

import scanpy as sc

In [2]:
n_cell = 5  # filter low exp gene: min cell expressed in this gene
dataset = 'AIBS_TENX'
std_cutoff = 0.05

In [3]:
# Parameters
cpu = 1
dataset = "AIBS_TENX"
group_name = "CorticalExc"
mem_gb = 1
n_cell = 5
std_cutoff = 0.05


## Select cells

In [4]:
cells = pd.read_csv('rna_cells.txt', index_col=0, header=None).index
cells.name = 'cell'
cells.size

1412319

## Get adata with basic feature selection

In [5]:
if dataset == 'AIBS_SMART':
    mcds_path = aibs.AIBS_SMART_ZARR_PATH
    gene_map = aibs.get_smart_gene_map()
elif dataset == 'AIBS_TENX':
    mcds_path = aibs.AIBS_TENX_V2_ZARR_PATH
    # AIBS TENX v2 has the same annotation as CEMBA mC geneslop2k-vm23
    # all using GENCDOE vm23 BICCN version
    gene_map = None
else:
    mcds_path = broad.BROAD_TENX_V2_ZARR_PATH
    gene_map = broad.get_tenx_gene_map()

In [6]:
rna_ds = MCDS.open(mcds_path, use_obs=cells, var_dim='gene')
rna_ds

<xarray.MCDS>
Dimensions:     (cell: 1412319, gene: 32285)
Coordinates:
  * cell        (cell) <U37 'L8TX_190430_01_A08-GTCGTAAGTGTTAACC-1' ... 'L8TX...
  * gene        (gene) <U18 'ENSMUSG00000051951' ... 'ENSMUSG00000095041'
    gene_count  (cell) float32 4.161e+03 5.638e+03 ... 5.299e+03 4.487e+03
    gene_name   (gene) <U16 'Xkr4' 'Gm1992' ... 'AC234645.1' 'AC149090.1'
    umi_count   (cell) float32 1.121e+04 1.962e+04 ... 1.456e+04 1.119e+04
Data variables:
    gene_da     (cell, gene) float32 dask.array<chunksize=(5000, 32285), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  gene

In [7]:
mc_cef = pd.read_csv('mC.CEF.csv', header=None, index_col=0).squeeze()
rna_cef = pd.read_csv('RNA.CEF.csv', header=None, index_col=0).squeeze()
cef = mc_cef | rna_cef
cef = cef[cef].index
cef.name = 'gene'

In [8]:
if gene_map is None:
    assert (~cef.isin(rna_ds.get_index('gene'))).sum() == 0
    use_genes = rna_ds.get_index('gene').isin(cef)
else:
    gene_id_not_nan = ~rna_ds.get_index('gene').map(gene_map).isna()
    is_cef = rna_ds.get_index('gene').map(gene_map).isin(cef)
    use_genes = gene_id_not_nan & is_cef
use_genes.sum()

10842

In [9]:
loading_chunk = int(cells.size // 5000 / 20) * 5000
loading_chunk = max(20000, loading_chunk)

# ~20 min to load 4M
adata = rna_ds.get_count_adata(da_name='gene_da', use_vars=use_genes, 
                               loading_chunk=loading_chunk)

Loading chunk 0-70000/1412319


Loading chunk 70000-140000/1412319


Loading chunk 140000-210000/1412319


Loading chunk 210000-280000/1412319


Loading chunk 280000-350000/1412319


...

Loading chunk 1120000-1190000/1412319


Loading chunk 1190000-1260000/1412319


Loading chunk 1260000-1330000/1412319


Loading chunk 1330000-1400000/1412319


Loading chunk 1400000-1412319/1412319


In [10]:
if gene_map is not None:
    # cleanup gene metadata of RNA
    adata.var_names = adata.var_names.map(gene_map)
    assert adata.var_names.duplicated().sum() == 0

    gene_meta = mm10.get_gene_metadata(annot_version = 'GENCODE_vm22')
    for col in ['chrom', 'start', 'end']:
        adata.var[col] = gene_meta[col]
else:
    gene_meta = mm10.get_gene_metadata(annot_version = 'GENCODE_vm23')
    for col in ['chrom', 'start', 'end']:
        adata.var[col] = gene_meta[col]

In [11]:
adata

AnnData object with n_obs × n_vars = 1412319 × 10842
    obs: 'count', 'umi_count'
    var: 'name', 'chrom', 'start', 'end'

## Basic Feature Filtering

In [12]:
# no need to do this, already did in mC

# chroms = [
#     'chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9',
#     'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17',
#     'chr18', 'chr19'
# ]
#
# remove_chromosomes(adata, include_chromosomes=chroms)
# remove_black_list_region(adata, black_list_path=mm10.ENCODE_BLACKLIST_PATH)

In [13]:
if adata.n_obs > n_cell:
    filter_regions(adata, n_cell=n_cell, zscore_abs_cutoff=None)

10643 regions remained.


In [14]:
if adata.X.dtype != 'float32':
    adata.X = adata.X.astype('float32')

In [15]:
sc.pp.normalize_per_cell(adata)

In [16]:
sc.pp.log1p(adata)

In [17]:
adata.X = adata.X.toarray()
std_filter = adata.X.std(axis=0) > std_cutoff
adata._inplace_subset_var(std_filter)

In [18]:
# zero_center=False to prevent making dense matrix
sc.pp.scale(adata, zero_center=True)

In [19]:
adata.write_h5ad('rna_input.h5ad')

In [20]:
adata

AnnData object with n_obs × n_vars = 1412319 × 6231
    obs: 'count', 'umi_count', 'n_counts'
    var: 'name', 'chrom', 'start', 'end', 'mean', 'std'
    uns: 'log1p'